# Pre-Janesweather

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
impoxrt glob

import pickle
from datetime import datetime

In [2]:
path = '/home/student.unimelb.edu.au/aborowiak/Desktop/Code/Current Project/Stations/MJO/RMM_meta.csv'

In [141]:
date_list = []
phase_list = []
amp_list = []


with open(path) as RMM_meta:
    lines = RMM_meta.readlines()
    titles = lines[0].split()
    for row_num in np.arange(1,len(lines)):
        line = lines[row_num].split()
        
        year = line[0]
        month = line[1]
        day = line[2]

        date = pd.to_datetime(year +'-'+ month +'-'+ day, format = '%Y-%m-%d')
        date_list.append(date)

        
        phase = line[5]
        phase_list.append(phase)
        amp = line[6]
        amp_list.append(amp)


RMM = pd.DataFrame()
RMM['Date'] = date_list
RMM['Phase'] = phase_list
RMM['Phase'] = RMM['Phase'].astype(float)
RMM['Amplitude'] = amp_list
RMM['Amplitude'] = RMM['Amplitude'].astype(float)
RMM = RMM.set_index('Date')

In [142]:
RMM.head()

,Phase,Amplitude
Date,,
1974-06-01,5.0,2.02948
1974-06-02,5.0,1.89729
1974-06-03,5.0,1.86476
1974-06-04,5.0,1.83092
1974-06-05,5.0,2.03326


In [143]:
out_file = open('RMM.pickle', 'wb')
pickle.dump(RMM, out_file)
out_file.close()

# Janesweather

In [11]:



%matplotlib inline
import io
import os
import sys
import calendar
import numpy as np
import pandas as pd
import xarray as xr
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import urllib
from matplotlib import pyplot as plt
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)
# import svgutils.compose as sc
from IPython.display import SVG

from datetime import datetime
from dateutil.parser import parse as dparse
import calendar
from IPython.display import HTML, Image, IFrame

import pickle

In [3]:
url = 'http://www.bom.gov.au/climate/mjo/graphics/rmm.74toRealtime.txt'
user_agent = 'Mozilla/5.0 (X11; Linux x86_64; rv:60.0) Gecko/20100101 Firefox/60.0'
headers={'User-Agent':user_agent}
request=urllib.request.Request(url,None,headers)
response = urllib.request.urlopen(request)
data = response.read()
csv = io.StringIO(data.decode('utf-8'))

rmm_df = pd.read_csv(csv, sep=r'\s+', header=None, skiprows=2,
    usecols=[0,1,2,3,4,5,6,7], names=['year', 'month', 'day','RMM1','RMM2', 'phase', 'amplitude', 'origin'])

In [4]:
index = pd.to_datetime(rmm_df.loc[:,['year','month','day']])
rmm_df.index = index
rmm_df.head()

,year,month,day,RMM1,RMM2,phase,amplitude,origin
1974-06-01,1974,6,1,1.63447,1.20304,5,2.02948,Final_value:__OLR_&_NCEP_winds
1974-06-02,1974,6,2,1.60289,1.01512,5,1.89729,Final_value:__OLR_&_NCEP_winds
1974-06-03,1974,6,3,1.51625,1.08551,5,1.86476,Final_value:__OLR_&_NCEP_winds
1974-06-04,1974,6,4,1.50981,1.03573,5,1.83092,Final_value:__OLR_&_NCEP_winds
1974-06-05,1974,6,5,1.55906,1.30518,5,2.03326,Final_value:__OLR_&_NCEP_winds


In [13]:
rmm_df_2 = rmm_df.loc[:,['phase', 'amplitude','RMM1','RMM2']]
rmm_df_2.index = rmm_df.index
rmm_df_2.head()

,phase,amplitude,RMM1,RMM2
1974-06-01,5,2.02948,1.63447,1.20304
1974-06-02,5,1.89729,1.60289,1.01512
1974-06-03,5,1.86476,1.51625,1.08551
1974-06-04,5,1.83092,1.50981,1.03573
1974-06-05,5,2.03326,1.55906,1.30518


In [14]:
rmm_observed = rmm_df_2.to_xarray()
rmm_observed = rmm_observed.rename({'index':'time','RMM1':'rmm1','RMM2':'rmm2'})
rmm_observed

<xarray.Dataset>
Dimensions:    (time: 16592)
Coordinates:
  * time       (time) datetime64[ns] 1974-06-01 1974-06-02 ... 2019-11-03
Data variables:
    phase      (time) int64 5 5 5 5 5 6 6 6 6 7 6 5 ... 2 3 3 3 3 4 4 4 5 5 5 5
    amplitude  (time) float64 2.029 1.897 1.865 1.831 ... 1.845 2.149 2.367
    rmm1       (time) float64 1.634 1.603 1.516 1.51 ... 1.382 1.788 1.978 2.061
    rmm2       (time) float64 1.203 1.015 1.086 1.036 ... 0.4561 0.8404 1.165

In [8]:
# rmm_observed.to_netcdf('rmm_obs.nc')

In [15]:
%%writefile rmm_observed.txt

The .nc version of the file is from the bureau of meterolgy website

Overwriting rmm_observed.txt


In [16]:
out_file = open('rmm_observe.pickle', 'wb')
pickle.dump(rmm_df_2, out_file)
out_file.close()